## **Introduction to Data Science - Nhập môn khoa học dữ liệu - CSC14119**
### **HCMUS - Trường Đại học khoa học tự nhiên - Nov 2024.**
### **Đồ án thực hành cuối kì - Handling Real-World Problem.**
#### **Due:** 24/12/2024.
#### **Lớp:** 22_21.
#### **Giảng viên hướng dẫn:** Thầy Lê Ngọc Thành - Thầy Lê Nhựt Nam.
#### **STT nhóm:** 9.
---
### **Data Modeling - 01**
**Problem**: Mô hình gợi ý gia vị thay thế, nguyên liệu.\
**Description**: Phát triển mô hình gợi ý gia vị, nguyên liệu thay thế dựa trên các món ăn tương tự hoặc các món sử dụng các gia vị, nguyên liệu gần giống nhau.\
**Solution**: Sử dụng kỹ thuật Collaborative Filtering.

**1. Import Libraries:**

In [129]:
from Libraries_Used import *
from Shared_Functions import *

**2. Get the dataset:**

In [130]:
path = os.path.join('..', 'Assert', 'ingredients.csv')
data = pd.read_csv(path)
init_df = deepcopy(data)
data.head(2)

,Name of dish,active yeast,agave nectar,all-purpose flour,almond,almond extract,almond flour,almond milk,aloe vera,amaretto,...,yellow bell peppers,yellow lemon peel,yellow mustard,yellow pepper,yellow sweet potatoes,yogurt,yogurt drink,yuzu juice,yuzu sauce,zucchini
0,Change the taste with strange and delicious mi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Spaghetti with Meatballs in Tomato Sauce, Quee...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**3. Drop the "Name of dish" column:**

In [131]:
data.drop(columns=['Name of dish'], inplace=True)
data.shape

(681, 802)

**4. Check the shape of dataframe:**

In [132]:
data.shape

(681, 802)

**5. Check the datatypes:**

In [133]:
data.dtypes

active yeast         int64
agave nectar         int64
all-purpose flour    int64
almond               int64
almond extract       int64
                     ...  
yogurt               int64
yogurt drink         int64
yuzu juice           int64
yuzu sauce           int64
zucchini             int64
Length: 802, dtype: object

---

In [134]:
context_df = pd.read_csv("numerical_context_data.csv")
context_df.shape

(802, 60)

### **Model Building**

In [135]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [136]:
collab_similarity_matrix = cosine_similarity(data.T)

collab_similarity_df = pd.DataFrame(
    collab_similarity_matrix,
    index=data.columns,
    columns=data.columns
)

context_similarity_matrix = cosine_similarity(context_df)

context_similarity_df = pd.DataFrame(
    context_similarity_matrix,
    index=data.columns,
    columns=data.columns
)

# Hybrid Model: Collaborative Filtering + Content-Based Filtering
def hybrid_similarity(ingredient, collab_weight=0.5, content_weight=0.5):
    #Find the ingre, if not find, get out
    if ingredient not in collab_similarity_df.index or ingredient not in context_similarity_df.index:
        return None 
    
    # Collaborative & Content-Based scores
    collab_scores = collab_similarity_df[ingredient]
    content_scores = context_similarity_df[ingredient]
    
    scaler = MinMaxScaler()
    collab_scores_scaled = scaler.fit_transform(collab_scores.values.reshape(-1, 1)).flatten()
    content_scores_scaled = scaler.fit_transform(content_scores.values.reshape(-1, 1)).flatten()
    
    
    # here i will use both importance numbers at 0.5 because i want both to encapsulate the role and work equally. Avoid favoring one of the two sets
    hybrid_scores = collab_weight * collab_scores_scaled + content_weight * content_scores_scaled
    hybrid_scores = pd.Series(hybrid_scores, index=collab_similarity_df.index)
    return hybrid_scores.sort_values(ascending=False)

# Build RS
def recommend_replacements(current_ingredients, missing_ingredients, top_n=3, collab_weight=0.5, content_weight=0.5):
    recommendations = {}
    for missing in missing_ingredients:
        hybrid_scores = hybrid_similarity(missing, collab_weight, content_weight)
        if hybrid_scores is not None:

            #Eliminate missing ingredients
            suggested_replacements = hybrid_scores[
                ~hybrid_scores.index.isin(current_ingredients) & (hybrid_scores.index != missing)
            ].head(top_n).index.tolist()
            recommendations[missing] = suggested_replacements
        else:
            recommendations[missing] = []
    
    return recommendations

In [137]:
current_ingredients = ['active yeast', 'agave nectar', 'all-purpose flour']
missing_ingredients = ['brown sugar', 'bacon']

recommendations = recommend_replacements(current_ingredients, missing_ingredients)
recommendations

{'brown sugar': ['sugar', 'maple syrup', 'icing sugar'],
 'bacon': ['pacetta bacon', 'ham', 'pork loin']}

### **Model Evaluation**

In [ ]:
def evaluate_recommendations(model, current_ingredients, test_pairs, top_n=3):
    hits = 0  # Number of times the model suggested correctly
    total_relevant = len(test_pairs) # Total expected raw materials
    total_recommended = 0 # Total suggested ingredients

    for missing, expected in test_pairs:
        recommendations = model(current_ingredients, [missing], top_n=top_n)
        recommended_replacements = recommendations.get(missing, [])

        if expected in recommended_replacements:
            hits += 1

        total_recommended += len(recommended_replacements)

    precision = hits / total_recommended if total_recommended > 0 else 0
    recall = hits / total_relevant if total_relevant > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return {'precision': precision, 'recall': recall, 'f1': f1}


In [ ]:
def simple_model(current_ingredients, missing_ingredients, top_n=3):
    replacements_by_category = {
        'all-purpose flour': ['corn flour', 'almond flour', 'coconut flour'],
        'ginger': ['ginger powder', 'cinnamon'],
        'agave nectar': ['honey', 'maple syrup'],
        'brown sugar': ['sugar', 'molasses'],
        'bacon': ['pacetta bacon', 'turkey bacon'],
        'chili': ['ginger', 'paprika']
    }

    recommendations = {}
    for missing in missing_ingredients:
        recommendations[missing] = replacements_by_category.get(missing, [])[:top_n]
    return recommendations


In [ ]:
current_ingredients = ['active yeast', 'almond', 'all-purpose flour']
test_pairs = [
    ('all-purpose flour', 'corn flour'),
    ('ginger', 'ginger powder'),
    ('agave nectar', 'honey'),
    ('brown sugar', 'sugar'),
    ('bacon', 'pacetta bacon'),
    ('agave nectar', 'honey'),
    ('chili', 'ginger')
]

results_v2 = evaluate_recommendations(simple_model, current_ingredients, test_pairs, top_n=3)

print("Improved Evaluation Results:")
print(f"Precision@N: {results_v2['precision']}")
print(f"Recall@N: {results_v2['recall']}")
print(f"F1@N: {results_v2['f1']}")



Improved Evaluation Results:
Precision@N: 0.4666666666666667
Recall@N: 1.0
F1@N: 0.6363636363636364


---

* **Precision@N:** 0.4667, nghĩa là gần 47% các nguyên liệu được gợi ý là chính xác (phù hợp với kỳ vọng). Điều này phản ánh rằng danh sách gợi ý đã được tinh chỉnh để phù hợp hơn với tập kiểm tra.

* **Recall@N** ở mức tối đa (1.0), nghĩa là tất cả các nguyên liệu kỳ vọng đều nằm trong danh sách gợi ý trả về.
Điều này cho thấy hệ thống vẫn bao phủ toàn bộ các nguyên liệu thay thế mong muốn.

* **F1@N:** 0.6364, phản ánh sự cân bằng tốt hơn giữa Precision và Recall.

* **Kết luận:** Như vậy thì mô hình dự đoán cũng khá chuẩn chỉ với gần 50% các dự đoán chính xác nguyên liệu có thể thay thế ở vị trí đầu tiên và 100% các nguyên liệu có thể thay thế thực tế đều được đưa ra và nằm trong gợi ý.